## **Camada Silver**
Nesta camada, verifiquei valores nulos e duplicatas, padronizei os nomes das colunas e, ao final, uni as bases de exportação e importação, salvando o resultado no **Delta Lake**.


In [0]:
from pyspark.sql.functions import col, sum 

In [0]:
spark.sql("""show tables  in bronze """).show() 


In [0]:
# Path dos dados camada Bronze
path_exp24 = 'dbk_brigadeiro.bronze.tb_importacao25'
path_exp25 = 'dbk_brigadeiro.bronze.tb_exportacao25'

path_imp24  = 'dbk_brigadeiro.bronze.tb_importacao24'
path_imp25 = 'dbk_brigadeiro.bronze.tb_importacao25'

path_produtos = 'dbk_brigadeiro.bronze.tb_produtos'
path_bloco =   'dbk_brigadeiro.bronze.tb_blocoeconomico'
path_paises = 'dbk_brigadeiro.bronze.tb_paises'
path_municipios = 'dbk_brigadeiro.bronze.tb_municipios'
path_urf = 'dbk_brigadeiro.bronze.tb_urf'
path_via = 'dbk_brigadeiro.bronze.tb_via'

In [0]:
tb_exportacao24 = spark.read.table(path_exp24)
tb_exportacao25 = spark.read.table(path_exp25)

tb_importacao24 = spark.read.table(path_imp24)
tb_importacao25 = spark.read.table(path_imp25)


In [0]:
tb_bloco = spark.read.table(path_bloco)
tb_paises = spark.read.table(path_paises)
tb_municipio = spark.read.table(path_municipios)
tb_produtos = spark.read.table(path_produtos)
tb_urf = spark.read.table(path_urf)
tb_via = spark.read.table(path_via)

**Limpeza dos dados**

In [0]:
display(tb_importacao24.limit(2))

In [0]:
display(tb_exportacao24.limit(2))

**Verificar Nulos**

In [0]:
# spark.sql("""show catalogs """).show()

# spark.sql("""describe CATALOG  dbk_brigadeiro """).show()

spark.sql("""show tables  in bronze """).show() 


In [0]:
display(tb_exportacao24.select([sum(col(c).isNull().cast("int")).alias(c) for c in tb_exportacao24.columns]).show())

In [0]:
# display(tb_exportacao25.select([sum(col(c).isNull().cast("int")).alias(c) for c in tb_exportacao25.columns]).show())

In [0]:
# display(tb_importacao24.select([sum(col(c).isNull().cast("int")).alias(c) for c in tb_importacao24.columns]).show())

In [0]:
# display(tb_importacao25.select([sum(col(c).isNull().cast("int")).alias(c) for c in tb_importacao25.columns]).show())

**Padronizar nome de colunas**

In [0]:
# tb_bloco.show(2)

In [0]:
# tb_via.show(2)

In [0]:
# display(tb_produtos)

In [0]:
tb_via  = (tb_via.withColumnRenamed('CO_VIA', 'COD_VIA')
        .withColumnRenamed('NO_VIA', 'VIA')
)

In [0]:
tb_bloco = tb_bloco.drop('NO_BLOCO_ING', 'NO_BLOCO_ESP')
tb_bloco = (tb_bloco.withColumnRenamed('CO_PAIS', 'COD_PAIS')
        .withColumnRenamed('CO_BLOCO', 'COD_BLOCO')
        .withColumnRenamed('NO_BLOCO', 'BLOCO')
        )

In [0]:
tb_paises = tb_paises.drop('CO_PAIS_ISON3', 'NO_PAIS_ING', 'NO_PAIS_ESP')
tb_paises = (
    tb_paises.withColumnRenamed('CO_PAIS', 'COD_PAIS')
    .withColumnRenamed('CO_PAIS_ISOA3', 'COD_PAIS_ISOA3')
    .withColumnRenamed('NO_PAIS', 'PAIS')
)

In [0]:
tb_municipio = tb_municipio.drop('NO_MUN_MIN', 'CO_MUN_GEO')
tb_municipio = (
    tb_municipio.withColumnRenamed('NO_MUN', 'CIDADE')
    .withColumnRenamed('SG_UF', 'UF')
)

In [0]:
tb_municipio.show(2)

In [0]:
tb_produtos = tb_produtos.drop('CO_SH6', 'CO_PPE','CO_PPI', 'CO_FAT_AGREG', 'CO_CUCI_ITEM', 'CO_CGCE_N3', 'CO_SIIT', 'CO_ISIC_CLASSE',
                               'CO_EXP_SUBSET', 'NO_NCM_ESP', 'NO_NCM_ING')
tb_produtos = (
    tb_produtos.withColumnRenamed('CO_NCM', 'COD_PRODUTO')
    .withColumnRenamed('CO_UNID', 'COD_UNID')
    .withColumnRenamed('NO_NCM_POR', 'PRODUTO')
)

tb_produtos = tb_produtos.withColumn('COD_PRODUTO', col('COD_PRODUTO').cast('int'))

In [0]:
tb_urf = (
    tb_urf.withColumnRenamed('CO_URF', 'COD_URF')
    .withColumnRenamed('NO_URF', 'URF')
)

In [0]:
def renomear_columns_EXP(tb_exportacao24, tb_exportacao25):
        colunas_EXP = ['COD_ANO','COD_MES', 'COD_PRODUTO','COD_UNID','COD_PAIS','UF_O_PRODUTO','COD_VIA',
                 'COD_URF','QTD_ESTAT','KG_LIQUIDO','VL_FOB', 'VL_FRETE', 'VL_SEGURO']

        coluna_EXP_ANT = ['CO_ANO','CO_MES', 'CO_NCM', 'CO_UNID','CO_PAIS', 'SG_UF_NCM', 'CO_VIA',
                      'CO_URF', 'QT_ESTAT', 'KG_LIQUIDO', 'VL_FOB', 'VL_FRETE','VL_SEGURO']
   
        for i in range(len(tb_exportacao24.columns)):
            tb_exportacao24 = tb_exportacao24.withColumnRenamed(coluna_EXP_ANT[i], colunas_EXP[i])
            tb_exportacao25 = tb_exportacao25.withColumnRenamed(coluna_EXP_ANT[i], colunas_EXP[i])

        return tb_exportacao24, tb_exportacao25
        


In [0]:
def renomear_columns_IMP(tb_importacao24, tb_importacao25):

    colunas_IMP = ['COD_ANO','COD_MES','COD_PRODUTO','COD_UNID','COD_PAIS', 'UF_O_PRODUTO', 'COD_VIA',
                        'COD_URF', 'QTD_ESTAT', 'KG_LIQUIDO', 'VL_FOB', 'VL_FRETE', 'VL_SEGURO']
    
    coluna_IMP_ANT = ['CO_ANO','CO_MES','CO_NCM','CO_UNID','CO_PAIS', 'SG_UF_NCM', 'CO_VIA',
                        'CO_URF', 'QT_ESTAT', 'KG_LIQUIDO', 'VL_FOB', 'VL_FRETE', 'VL_SEGURO']
     

    for i in range(len(tb_importacao24.columns)):
        tb_importacao24 = tb_importacao24.withColumnRenamed(coluna_IMP_ANT[i], colunas_IMP[i])
        tb_importacao25 = tb_importacao25.withColumnRenamed(coluna_IMP_ANT[i], colunas_IMP[i])

    return tb_importacao24, tb_importacao25

In [0]:
tb_exportacao24, tb_exportacao25 = renomear_columns_EXP(tb_exportacao24, tb_exportacao25)

In [0]:
tb_importacao24, tb_importacao25 =renomear_columns_IMP(tb_importacao24, tb_importacao25)

In [0]:
tb_exportacao24 = tb_exportacao24.dropDuplicates()
tb_exportacao25 = tb_exportacao25.dropDuplicates()

tb_importacao24 = tb_importacao24.dropDuplicates()
tb_importacao25 = tb_importacao25.dropDuplicates()

**Salvando no Delta Lake**

In [0]:
(tb_importacao24.repartition(8).write.format('delta').mode('overwrite')
 .saveAsTable('dbk_brigadeiro.silver.tb_importacao24')
)

(tb_importacao25.repartition(8).write.format('delta').mode('overwrite')
 .saveAsTable('dbk_brigadeiro.silver.tb_importacao25'))

In [0]:
(tb_exportacao24.repartition(8).write.format('delta').mode('overwrite')
 .saveAsTable('dbk_brigadeiro.silver.tb_exportacao24'))

(tb_exportacao25.repartition(8).write.format('delta').mode('overwrite')
 .saveAsTable('dbk_brigadeiro.silver.tb_exportacao25'))

In [0]:
tb_bloco.write.format('delta').mode('overwrite').saveAsTable('dbk_brigadeiro.silver.tb_bloco')
tb_municipio.write.format('delta').mode('overwrite').saveAsTable('dbk_brigadeiro.silver.tb_municipio')
tb_paises.write.format('delta').mode('overwrite').saveAsTable('dbk_brigadeiro.silver.tb_paise')
tb_produtos.write.format('delta').mode('overwrite').saveAsTable('dbk_brigadeiro.silver.tb_produto')
tb_urf.write.format('delta').mode('overwrite').saveAsTable('dbk_brigadeiro.silver.tb_urf')
tb_via.write.format('delta').mode('overwrite').saveAsTable('dbk_brigadeiro.silver.tb_via')

In [0]:
%sql
-- show tables in etl_comercioexterior